# Trialing the OCLC Worldcat API

Notes on this process in my OneNote for now

In [ ]:
import sys
if "../" not in sys.path:
    sys.path.append("../")
import os
import io
import sys
from urllib.parse import quote
from dotenv import load_dotenv
from pymarc import marcxml, Record
import requests

import bookops_worldcat as bw

In [ ]:
load_dotenv()
client_id = os.environ["CLIENT_ID"]
client_secret = os.environ["CLIENT_SECRET"]

## Metadata API

Have to use access token authorisation but return values can be xml

In [ ]:
scope = "WorldCatMetadataAPI"
auth_url = f"https://oauth.oclc.org/token?grant_type=client_credentials&scope={scope}"
bib_headers = {"Accept":"application/json"}
oclc_headers = {"Accept":"application/marcxml+xml"}

In [ ]:
auth = requests.post(auth_url, headers=bib_headers, auth=(client_id, client_secret))
token = auth.json()["access_token"]
bib_headers["Authorization"] = f"Bearer {token}"
oclc_headers["Authorization"] = f"Bearer {token}"

bib_headers, oclc_headers

In [ ]:
auth.json()

In [ ]:
# query = 'ti="Feng Ling Du" AND au="Duanmu (Hongliang)"'
seventy_query = f'ti:"FEI LONG QUAN ZHUAN" and au:"WU (Xuan)"'

In [ ]:
metadata_url = "https://metadata.api.oclc.org/worldcat/"
search_brief_bibs = "search/brief-bibs?q="
get_record_by_oclc_num = "manage/bibs/"

ti_au_search_url = metadata_url + search_brief_bibs + quote(seventy_query) + "&orderBy=bestMatch"
oclc_search_stem = metadata_url + get_record_by_oclc_num

In [ ]:
ti_au_search_url

In [ ]:
# brief_bib_search_result = requests.get(ti_au_search_url.replace("%20", "+"), headers=bib_headers)

In [ ]:
first_50 = requests.get(ti_au_search_url+"&limit=50", headers=bib_headers)

In [ ]:
first_50_oclc = [x["oclcNumber"] for x in first_50.json()["briefRecords"]]

In [ ]:
second_20 = requests.get(ti_au_search_url + "&offset=52&limit=1", headers=bib_headers)

In [ ]:
len(first_50.json()["briefRecords"]), len(second_20.json()["briefRecords"])

In [ ]:
second_20_oclc = [x["oclcNumber"] for x in second_20.json()["briefRecords"]]
second_20_oclc

In [ ]:
[first_50_oclc.index(x) for x in second_20_oclc if x in first_50_oclc]

In [ ]:
first_50_oclc[-10:]

In [ ]:
brief_bib_search_result.json()["briefRecords"][0]

In [ ]:
oclc_search_stem + brief_bib_search_result.json()["briefRecords"][0]["oclcNumber"]

In [ ]:
marc_xml = {
    int(x["oclcNumber"]): requests.get(oclc_search_stem + x["oclcNumber"], headers=oclc_headers)
    for x in brief_bib_search_result.json()["briefRecords"]
}

In [ ]:
marc_xml[23921305]

In [ ]:
[x["oclcNumber"] for x in brief_bib_search_result.json()["briefRecords"]]

In [ ]:
print(marcxml.parse_xml_to_array(io.StringIO(marc_xml[23921305].text))[0])

In [ ]:
token = bw.WorldcatAccessToken(
    key=client_id,
    secret=client_secret,
    scopes=["WorldCatMetadataAPI"],
    principal_id="unused",
    principal_idns="unused"
)

In [ ]:
session = bw.MetadataSession(authorization=token)

In [ ]:
results = session.search_brief_bibs(q=query, inCatalogLanguage=None)

In [ ]:
results.json()

In [ ]:
full_bib = session.get_full_bib(results.json()["briefRecords"][0]["oclcNumber"])

In [ ]:
marcxml.parse_xml_to_array(io.StringIO(full_bib.text))[0]